# Basic usage of qmrExchange

### Import required libraries

In [1]:
from source.qmr_exchange import Exchange, Simulator
from source.Agents import RandomMarketTaker, NaiveMarketMaker
from datetime import datetime

### Declare basic parameters for the simulation
qmrExchange allows for simulating multiple tickers at once, for statistical arbitrage and high-frequency-trading simulations. In the present case, we simulate 1 week worth of 1 minute data (24/7 trading).


In [2]:
time_interval = 'minute'
tickers = ['XYZ']
episodes = 1000

### Instantiate a Simulator

In [3]:
sim = Simulator(time_unit = time_interval, episodes = episodes)
sim.exchange.create_asset(tickers[0])

### Add trading agents
- We add a naive market maker that creates both buy and sell orders in each period. It quotes buy and sell prices based on the last traded price and the specified spread percentage.
- We add a market taker that randomly buys and sells (based on the defined probabilities) on each period by means of market ordes (hence the word 'taker').

In [4]:

mm = NaiveMarketMaker(name='market_maker', tickers=tickers, aum=1_000, spread_pct=0.005, qty_per_order=4)
sim.add_agent(mm)

mt = RandomMarketTaker(name='market_taker', tickers=tickers, aum=1_000, prob_buy=.2, prob_sell=.2, qty_per_order=1,seed=42)
sim.add_agent(mt)

### Run the simulation

In [5]:
sim.run()

Retrieve all executed trades of our simulation

In [6]:
print(sim.trades.head(20).to_markdown())

| dt                  | ticker   |   qty |   price | buyer        | seller       |
|:--------------------|:---------|------:|--------:|:-------------|:-------------|
| 2022-01-01 00:00:00 | XYZ      |     1 |  100    | init_seed    | init_seed    |
| 2022-01-01 00:02:00 | XYZ      |     0 |   99.75 | market_maker | market_taker |
| 2022-01-01 00:03:00 | XYZ      |     0 |   99.5  | market_maker | market_taker |
| 2022-01-01 00:04:00 | XYZ      |     1 |   99.75 | market_taker | market_maker |
| 2022-01-01 00:08:00 | XYZ      |     1 |  100    | market_taker | market_maker |
| 2022-01-01 00:10:00 | XYZ      |     2 |   99.75 | market_maker | market_taker |
| 2022-01-01 00:11:00 | XYZ      |     1 |  100    | market_taker | market_maker |
| 2022-01-01 00:12:00 | XYZ      |     1 |   99.75 | market_maker | market_taker |
| 2022-01-01 00:13:00 | XYZ      |     0 |   99.5  | market_maker | market_taker |
| 2022-01-01 00:16:00 | XYZ      |     1 |   99.75 | market_taker | market_maker |
| 20

Group asset price in fixed 15 Minute OHLCV Bars

In [7]:
df_15min = sim.get_price_bars(ticker=tickers[0],bar_size='15Min')
print(df_15min.head(20).to_markdown())

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:02:00',
               '2022-01-01 00:03:00', '2022-01-01 00:04:00',
               '2022-01-01 00:08:00', '2022-01-01 00:10:00',
               '2022-01-01 00:11:00', '2022-01-01 00:12:00',
               '2022-01-01 00:13:00', '2022-01-01 00:16:00',
               ...
               '2022-01-01 16:16:00', '2022-01-01 16:20:00',
               '2022-01-01 16:24:00', '2022-01-01 16:29:00',
               '2022-01-01 16:31:00', '2022-01-01 16:33:00',
               '2022-01-01 16:34:00', '2022-01-01 16:35:00',
               '2022-01-01 16:37:00', '2022-01-01 16:40:00'],
              dtype='datetime64[ns]', name='dt', length=412, freq=None)
| dt                  |   open |   high |    low |   close |   volume |
|:--------------------|-------:|-------:|-------:|--------:|---------:|
| 2022-01-01 00:00:00 | 100    | 100    |  99.5  |   99.5  |        7 |
| 2022-01-01 00:15:00 |  99.75 | 100    |  99.25 |   99.5  |        8 |
| 2022-01-

Retrieve a dataframe of an agents holding at each period of time

In [8]:
mt_holdings = sim.get_portfolio_history('market_taker')
mm_holdings = sim.get_portfolio_history('market_maker')

print(mt_holdings.head(20).to_markdown())

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:02:00',
               '2022-01-01 00:03:00', '2022-01-01 00:04:00',
               '2022-01-01 00:08:00', '2022-01-01 00:10:00',
               '2022-01-01 00:11:00', '2022-01-01 00:12:00',
               '2022-01-01 00:13:00', '2022-01-01 00:16:00',
               ...
               '2022-01-01 16:16:00', '2022-01-01 16:20:00',
               '2022-01-01 16:24:00', '2022-01-01 16:29:00',
               '2022-01-01 16:31:00', '2022-01-01 16:33:00',
               '2022-01-01 16:34:00', '2022-01-01 16:35:00',
               '2022-01-01 16:37:00', '2022-01-01 16:40:00'],
              dtype='datetime64[ns]', name='dt', length=412, freq=None)
DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:02:00',
               '2022-01-01 00:03:00', '2022-01-01 00:04:00',
               '2022-01-01 00:08:00', '2022-01-01 00:10:00',
               '2022-01-01 00:11:00', '2022-01-01 00:12:00',
               '2022-01-01 00:13:00', '2022-01-01 00:1

### Plot the results

In [9]:
from source.helpers import plot_bars
df_15min = sim.get_price_bars(ticker=tickers[0], bar_size='15Min')
plot_bars(df_15min)

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:02:00',
               '2022-01-01 00:03:00', '2022-01-01 00:04:00',
               '2022-01-01 00:08:00', '2022-01-01 00:10:00',
               '2022-01-01 00:11:00', '2022-01-01 00:12:00',
               '2022-01-01 00:13:00', '2022-01-01 00:16:00',
               ...
               '2022-01-01 16:16:00', '2022-01-01 16:20:00',
               '2022-01-01 16:24:00', '2022-01-01 16:29:00',
               '2022-01-01 16:31:00', '2022-01-01 16:33:00',
               '2022-01-01 16:34:00', '2022-01-01 16:35:00',
               '2022-01-01 16:37:00', '2022-01-01 16:40:00'],
              dtype='datetime64[ns]', name='dt', length=412, freq=None)


In [10]:
import plotly.express as px
import pandas as pd

df_plot = pd.DataFrame()
df_plot['Market Maker'] = mm_holdings['aum']
df_plot['Market Taker'] = mt_holdings['aum']
fig = px.line(df_plot,labels={'variable':'Agents','value':'Assets Under Management','dt':'Date'})
fig.show()